In [35]:
from flask import Flask, request, jsonify
import redis
import csv
import msgpack
import pandas as pd
import numpy as np
import time
import argparse
import difflib
import json
import time
import pickle
from collections import defaultdict
from collections import deque
import tensorflow
from keras_preprocessing.text import tokenizer_from_json
from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship, NodeMatch, NodeMatcher
from pyjarowinkler import distance
import itertools

In [4]:
with open('../english_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [5]:
loaded_dict['test']

'trying something to find out about it'

In [7]:
def from_df_to_dict(df):
        first_column = df.iloc[:, 0].tolist()
        second_column = df.iloc[:, 1].tolist()
        #dict_object = my_dictionary()
        dict_object = defaultdict()
        for i in range(len(first_column)):
            dict_object[first_column[i]]=  second_column[i]
        
        return dict_object

In [8]:
vasu_df = pd.read_csv('../vasu_df.csv')

In [9]:
vasu_dict = from_df_to_dict(vasu_df)

In [67]:
small_dict = defaultdict(str)
words = ["name", "test", "field", "number", "best", "cook", "laptop", "dev", "production", "small"]
for i in range(len(words)):
    small_dict[words[i]] = f"{i}"

In [122]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

In [141]:
graph = Graph(uri, auth=(username, password))

In [69]:
def importDict_neo4j(dict_test):
    root_node = Node("business_modeler_terms", name="", description="")
    graph.create(root_node)

    for word, definition in dict_test.items():
        node = Node("business_modeler_terms", name=word, description=definition)
        graph.create(node)
        relationship = Relationship(root_node, "HAS", node)
        graph.create(relationship)

In [70]:
importDict_neo4j(small_dict)

In [19]:
def get_words_from_graph():
    matcher = NodeMatcher(graph)
    nodes = matcher.match("business_modeler_terms").where("_.name <> ''")
    return [node["name"] for node in nodes]

In [86]:
def search_node_in_graph(word):
    query = "MATCH (n:business_modeler_terms) WHERE toLower(n.name) = toLower($word) RETURN n.description AS description"
    result = graph.run(query, word=word).data()
    description = result[0]['description'] if result else None
    return description

In [131]:
search_node_in_graph("test")

'1'

In [36]:
def get_close_matches_icase(word, possibilities, *args, **kwargs):
    """ Case-insensitive version of difflib.get_close_matches """
    lword = word.lower()
    lpos = {}
    for p in possibilities:
        if p.lower() not in lpos:
            lpos[p.lower()] = [p]
        else:
            lpos[p.lower()].append(p)
    lmatches = difflib.get_close_matches(lword, lpos.keys(), *args, **kwargs)
    ret = [lpos[m] for m in lmatches]
    ret = itertools.chain.from_iterable(ret)
    return set(ret)

In [113]:
def fuzzy_search(word, cutoff, n):
    exact = search_node_in_graph(word)
    if exact == None:
        results = get_close_matches_icase(word, get_words_from_graph(), n=n, cutoff=cutoff)
        res = {result: (search_node_in_graph(result), difflib.SequenceMatcher(None, word.lower(), result.lower()).ratio()) for result in results}
        return dict(sorted(res.items(), key=lambda item: item[1][1], reverse=True))
    return {word: exact}

In [139]:
fuzzy_search("capt", 0.6, 10)

{'CAPTEST': ('capital', 0.7272727272727273), 'laptop': ('6', 0.6)}

In [159]:
def fuzzy_full_text_search(word, n):
    query = """
        CALL db.index.fulltext.queryNodes("businessGlossaryIndex", $word, {limit:$n}) YIELD node, score 
        RETURN node.name, node.description, score
    """
    parameters = {"word": word + "~", "n": n}
    return graph.run(query, parameters=parameters).data()


In [162]:
fuzzy_full_text_search("lest", 2)

[{'node.name': 'best', 'node.description': '4', 'score': 0.8276413679122925},
 {'node.name': 'test', 'node.description': '1', 'score': 0.8276413679122925}]

In [31]:
def update_description(word, new_description):
    query = "MATCH (n:business_modeler_terms {name: $word}) SET n.description = $new_description"
    graph.run(query, word=word, new_description=new_description)

In [32]:
update_description("test", "TEST NEW DESC")

In [29]:
def insert_node(word, description):
    node = Node("business_modeler_terms", name=word, description=description)
    graph.create(node)

    root_node = graph.nodes.match("business_modeler_terms", name="", description="").first()
    relationship = Relationship(root_node, "HAS", node)

    graph.create(relationship)

In [76]:
insert_node("CAPTEST", "capital")

In [73]:
def insert_dict(to_insert):
    for word, desc in to_insert.items():
        insert_node(word, desc)

In [74]:
another_small_dict = {"woo": "I1", "again": "I2", "last": "I3"}

In [75]:
insert_dict(another_small_dict)